### Import libraries

In [ ]:
import requests
import json
import re
from collections import Counter # descending sort
import gspread # Google Sheets
from oauth2client.service_account import ServiceAccountCredentials # Google Sheets

### Google Sheets authorisation

In [ ]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('#', scope) # path to the file with a key
client = gspread.authorize(creds)
sheet = client.open('#').get_worksheet(0) # the name of the worksheet

### Collecting the text corpus from every social group

In [ ]:
url = "https://api.vk.com/method/wall.get?"
query = {'access_token':'#', # VK API access_token
        'v':'5.103',
        'domain':'', # a group's domain
        'offset':0,
        'extended':0,
        'count':1}

r_first = requests.get(url,params=query)
q_first = r_first.json()

post_found = q_first['response']['count'] # total count of posts in a group

loops = post_found // 100 + 1 # offset

raw_posts = [] # a data set with all the texts

#print('found:',post_found)

for i in range(loops): # collect every 100 posts
    
    offset = i*100
    second_query = {'access_token':'', # VK API access_token
                    'v':'5.103',
                    'domain':'#', # a group's domain
                    'offset':offset,
                    'extended':0,
                    'count':100}

    r_second = requests.get(url,params=second_query)
    q_second = r_second.json()
    
    post_range = len(q_second['response']['items'])
    
    for k in range(0,post_range):
        raw_item = {'from_id':q_second['response']['items'][k]['from_id'],'post_id':q_second['response']['items'][k]['id'],'date':q_second['response']['items'][k]['date'],'text':q_second['response']['items'][k]['text']} 
        raw_posts.append(raw_item)
        
# check if all the posts are collected

# print('Posts in total:', len(raw_posts), 'should be:', post_found)

### Finding tokens in every text and transferring data to Google Sheets

##### Note: the list 'plust_dict' contained all the stems and their regular expressions

In [ ]:
index = 0 # a raw in Google Sheets

for d in plus_dict: # check every regular expresion
    
    results = [] # a list with the found tokens
    token_search = d['regex'] # a single regEX

    for t in raw_posts: # checking the text corpus
        
        tokens_found = re.findall(token_search, t['text'])

        if len(tokens_found) != 0: # if found, save it in results
            dict_item = {'from_id':t['from_id'],'post_id':t['post_id'],'date':t['date'],'text':t['text'],'tokens':tokens_found} 
            results.append(dict_item)

    invariants = [] # a list with all the word forms
    
    for l in results:
        for s in l['tokens']:
            invariants.append(s.lower()) # lowercase
    report = Counter(invariants).most_common() # descending sort

    results.append({'words':len(invariants),'n_invariants':len(report), 'invariants':dict(report)})

    if len(results) > 1:
        with open(f"#/{d['word']}.json", 'w', encoding='utf-8') as outfile: # save data as a json file
            json.dump(results, outfile, ensure_ascii=False)

        row = [d['word'],results[-1]['words'],results[-1]['n_invariants'],
               f"{results[-1]['invariants']}".strip("{}"),
               len(results)-1,len(raw_posts),"#","#",d['regex']] # add a raw in Google Sheets

        sheet.insert_row(row, index)
        
        # print(index, len(results)-1)
        
        index = index + 1 # go to the next row

    else:
        row = [d['word'],0,0,'none',0,len(raw_posts),"#","#",d['regex']] # if a stem is not found, add an emty row
        sheet.insert_row(row, index)
        
       # print(index, len(results)-1)
    
        index = index + 1 # go to the next row